# Manentia's DICOM to NIfTI and NIfTI to DICOM conversion Task


Complete the Python code of the given two classes.

`Python version >= 3.10`

### Download two test samples from here:
1. https://drive.google.com/file/d/1PqTEEFc0YbV7iCF8rDYRltE-Y4h95V-E/view?usp=sharing
2. https://drive.google.com/file/d/1ifdNew-8qXGKASZ58KIB5OHde89yEUB2/view?usp=sharing

Note: In sample 2, there are multiple series. Choose only one with the maximum number of slices to pass to DICOMToNIfTI(series_path) argument

In [5]:
# !pip install pydicom
%pip install pylibjpeg

In [8]:
from attrs import define, field, validators
from __future__ import annotations
from pathlib import Path
from pydicom import dcmread
import numpy as np
import nibabel as nib
import datetime
import pylibjpeg

@define
class DICOMToNIfTI:
    series_paths: list[str | Path] = field()
    save_dirpath: str | Path = field(default="mai_output", converter=Path)

    def convert(self):
        # Load CT series
        ds_list = []
        for filepath in self.series_paths:
            ds = dcmread(filepath)
            print(f"{ds.get('ImagePositionPatient')[-1] = }")
            ds_list.append(ds)

        # TODO: Sort the CT slices using ImagePositionPatient
        ds_list.sort(key=lambda ds: ds.ImagePositionPatient[-1])


        volm, meta_list = [], []
        for ds in ds_list:
            volm.append(ds.pixel_array)
            meta_list.append({
                'patient_id': ds.get('PatientID'),
                'study_uid': ds.get('StudyInstanceUID'),
                'series_uid': ds.get('SeriesInstanceUID'),
                'modality': ds.get('Modality'),
                'bodypart': ds.get('BodyPartExamined'),
                'rows': ds.get('Rows'),
                'cols': ds.get('Columns'),
                'imagepositionpatient': ds.get('ImagePositionPatient'), # if 'ImagePositionPatient' in ds else None,
                'transfer_syntax_uid': ds.file_meta.TransferSyntaxUID,
                'samples_per_pixel': ds.get('SamplesPerPixel'),
                'bits_allocated' : ds.get('BitsAllocated'),
                'pixel_representation' :ds.get('PixelRepresentation'),
            })
        
        # TODO: Convert to NIfTI
        volm_np = np.stack(volm, axis=-1)
        affine = np.eye(4)
        nifti_img = nib.Nifti1Image(volm_np, affine)
        
        save_dirpath = Path(self.save_dirpath)
        save_dirpath.mkdir(parents=True, exist_ok=True)
        nifti_filepath = save_dirpath / datetime.datetime.now().strftime("converted_volume_%Y-%m-%d_%H-%M-%S.nii")
        nib.save(nifti_img, nifti_filepath)

        path_to_nifti_file = nifti_filepath
        return path_to_nifti_file, meta_list

In [10]:
# sample 1
series_path = Path("3000566.000000-NA-03192")
paths_list = list(series_path.glob("*.dcm"))
path_to_nifti_file, meta_list = DICOMToNIfTI(paths_list).convert()

ValueError: need at least one array to stack

In [3]:
# sample 2 - series with max no of slices
import SimpleITK as sitk

def get_best_series_path(dicom_dir):
    def count_slices(series_file_names):
        return len(series_file_names)
    
    series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(dicom_dir)

    max_slices = 0
    best_series_file_names = []

    for series_ID in series_IDs:
        series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(dicom_dir, series_ID)
        num_slices = count_slices(series_file_names)
        #print(f"{series_ID} : {num_slices} ")
        if num_slices > max_slices:
            max_slices = num_slices
            best_series_file_names = series_file_names

    return best_series_file_names

series_path = 'D:\\manentia_task\\JANAK_P'
paths_list = get_best_series_path(series_path)
path_to_nifti_file, meta_list = DICOMToNIfTI(paths_list).convert()

ds.get('ImagePositionPatient')[-1] = '1269.0'
ds.get('ImagePositionPatient')[-1] = '1269.5'
ds.get('ImagePositionPatient')[-1] = '1270.0'
ds.get('ImagePositionPatient')[-1] = '1270.5'
ds.get('ImagePositionPatient')[-1] = '1271.0'
ds.get('ImagePositionPatient')[-1] = '1271.5'
ds.get('ImagePositionPatient')[-1] = '1272.0'
ds.get('ImagePositionPatient')[-1] = '1272.5'
ds.get('ImagePositionPatient')[-1] = '1273.0'
ds.get('ImagePositionPatient')[-1] = '1273.5'
ds.get('ImagePositionPatient')[-1] = '1274.0'
ds.get('ImagePositionPatient')[-1] = '1274.5'
ds.get('ImagePositionPatient')[-1] = '1275.0'
ds.get('ImagePositionPatient')[-1] = '1275.5'
ds.get('ImagePositionPatient')[-1] = '1276.0'
ds.get('ImagePositionPatient')[-1] = '1276.5'
ds.get('ImagePositionPatient')[-1] = '1277.0'
ds.get('ImagePositionPatient')[-1] = '1277.5'
ds.get('ImagePositionPatient')[-1] = '1278.0'
ds.get('ImagePositionPatient')[-1] = '1278.5'
ds.get('ImagePositionPatient')[-1] = '1279.0'
ds.get('ImagePositionPatient')[-1]

In [4]:
import nibabel as nib
from pydicom import FileDataset, Dataset

@define
class NIfTIToDICOM:
    path_to_nifti_file: str | Path = field(converter=Path)
    meta_list: list[dict] = field(factory=list)
    save_dirpath: str | Path = field(default="mai_output", converter=Path)
    

    def convert(self):
        volm = nib.load(self.path_to_nifti_file).get_fdata()
        assert volm.shape[0] == self.meta_list[0]['rows']
        assert volm.shape[1] == self.meta_list[0]['cols']
        assert volm.shape[2] == len(self.meta_list)
        
        self.save_dirpath.mkdir(parents=True, exist_ok=True)
        
        # TODO: Convert NIfTI to DICOM
        for i in range(volm.shape[2]):

            ds = FileDataset(
                filename_or_obj=str(self.save_dirpath / f'slice_{i + 1:04d}.dcm'),
                dataset=Dataset(), 
                file_meta=Dataset(), 
                preamble=b"\0" * 128
            )

            # Add necessary metadata
            ds.PatientID = self.meta_list[i]['patient_id']
            ds.StudyInstanceUID = self.meta_list[i]['study_uid']
            ds.SeriesInstanceUID = self.meta_list[i]['series_uid']
            ds.Modality = "CT"
            ds.BodyPartExamined = "CHEST"
            ds.Rows = volm.shape[0]
            ds.Columns = volm.shape[1]
            ds.ImagePositionPatient = self.meta_list[i]['imagepositionpatient']
            ds.file_meta.TransferSyntaxUID = self.meta_list[i]['transfer_syntax_uid']
            ds.SamplesPerPixel = self.meta_list[i]['samples_per_pixel'] 
            ds.BitsAllocated = self.meta_list[i]['bits_allocated']
            ds.PixelRepresentation = self.meta_list[i]['pixel_representation'] 

            
            # TODO: Save the DICOMs in `self.save_dirpath`
            ds.save_as(str(self.save_dirpath / f'slice_{i + 1:04d}.dcm'))

        return self.save_dirpath
    
saved_dirpath = NIfTIToDICOM(path_to_nifti_file, meta_list).convert()

C:\Users\Lekhashree\anaconda3\lib\site-packages\pydicom\filewriter.py:1054: UserWarning: All encapsulated (compressed) transfer syntaxes must use explicit VR little endian encoding for the dataset. Set 'FileDataset.is_little_endian = True' and 'FileDataset.is_implicit_VR = False' before saving
  warnings.warn(


# Verify new DICOMs

In [5]:
paths_list = list(saved_dirpath.glob("*.dcm"))
path_to_nifti_file, meta_list = DICOMToNIfTI(paths_list).convert()

C:\Users\Lekhashree\anaconda3\lib\site-packages\pydicom\filereader.py:353: UserWarning: Expected explicit VR, but found implicit VR - using implicit VR for reading
  warnings.warn(msg + f" - using {found_vr} VR for reading", UserWarning)


ds.get('ImagePositionPatient')[-1] = '1269.0'
ds.get('ImagePositionPatient')[-1] = '1269.5'
ds.get('ImagePositionPatient')[-1] = '1270.0'
ds.get('ImagePositionPatient')[-1] = '1270.5'
ds.get('ImagePositionPatient')[-1] = '1271.0'
ds.get('ImagePositionPatient')[-1] = '1271.5'
ds.get('ImagePositionPatient')[-1] = '1272.0'
ds.get('ImagePositionPatient')[-1] = '1272.5'
ds.get('ImagePositionPatient')[-1] = '1273.0'
ds.get('ImagePositionPatient')[-1] = '1273.5'
ds.get('ImagePositionPatient')[-1] = '1274.0'
ds.get('ImagePositionPatient')[-1] = '1274.5'
ds.get('ImagePositionPatient')[-1] = '1275.0'
ds.get('ImagePositionPatient')[-1] = '1275.5'
ds.get('ImagePositionPatient')[-1] = '1276.0'
ds.get('ImagePositionPatient')[-1] = '1276.5'
ds.get('ImagePositionPatient')[-1] = '1277.0'
ds.get('ImagePositionPatient')[-1] = '1277.5'
ds.get('ImagePositionPatient')[-1] = '1278.0'
ds.get('ImagePositionPatient')[-1] = '1278.5'
ds.get('ImagePositionPatient')[-1] = '1279.0'
ds.get('ImagePositionPatient')[-1]

AttributeError: Unable to convert the pixel data as the following required elements are missing from the dataset: PhotometricInterpretation, PixelData